In [1]:
import jupyter_core
import os
from ipywidgets import *
from IPython.display import HTML, display, HTML
from notebook.services.config import ConfigManager
from __future__ import print_function
from string import Template

In [2]:
cdir = os.path.join(jupyter_core.paths.jupyter_config_dir(), 'custom')
ndir = os.path.join(jupyter_core.paths.jupyter_config_dir(), 'nbconfig')
css_filename = os.path.join(cdir, 'custom.css')
cjs_filename = os.path.join(cdir, 'custom.js')
edit_filename = os.path.join(ndir, 'edit.json')
cm = ConfigManager()
doc_tools_path = 'notebook-extensions-master/calysto/document-tools/main'

In [3]:
def create_form(desc, options, var, cb):
    label_layout = Layout(flex='1 1 auto')
    control_layout = Layout(flex='0 1 10%')
    form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        border='solid 1px lightgray',
        justify_content='space-between',
        width='60%'
    )
    w = Dropdown(options=options, value=var, layout=control_layout)
    w.observe(lambda x: cb(x['new']), names='value')
    return Box([Label(value=desc, layout=label_layout), w], layout=form_item_layout)

In [4]:
edit_config = """{
  "Editor": {
    "codemirror_options": {
      "vimMode": %s,
      "keyMap": "%s"
    }
  }
}
"""

sublime_editor_txt = """/****** AUTOMATICALLY GENERATED BY NotebookSettings **************/

/* Warning: Anything you add here will get overwritten! */

/** KEYMAP sublime **/
require(["notebook/js/cell", "base/js/namespace", "codemirror/keymap/sublime"],
    function(cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
        cell.Cell.options_default.cm_config.vimMode = false;
        cell.Cell.options_default.cm_config.extraKeys["Tab"] = function(cm) {
            cm.execCommand(cm.getOption("indentWithTabs") ? "insertTab" : "insertSoftTab");
        };
        cell.Cell.options_default.cm_config.extraKeys["Ctrl-Enter"] = function(cm) {};
        var cells = IPython.notebook.get_cells();
        for(var c=0; c< cells.length ; c++){
            cells[c].code_mirror.setOption('keyMap', 'sublime');
            cells[c].code_mirror.setOption("vimMode",  false);
            cells[c].code_mirror.setOption("extraKeys", {
                "Ctrl-Enter": function(cm) {},
                "Tab": function(cm) {
                    cm.execCommand(cm.getOption("indentWithTabs") ? "insertTab" : "insertSoftTab");
                }
            });
        }
    }
);
"""

emacs_editor_txt = """/****** AUTOMATICALLY GENERATED BY NotebookSettings **************/

/* Warning: Anything you add here will get overwritten! */

/** KEYMAP emacs **/
require(["notebook/js/cell", "base/js/namespace", "codemirror/keymap/emacs"],
    function(cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'emacs';
        cell.Cell.options_default.cm_config.vimMode = false;
        var cells = IPython.notebook.get_cells();
        for(var c=0; c< cells.length ; c++){
            cells[c].code_mirror.setOption('keyMap', 'emacs');
            cells[c].code_mirror.setOption("vimMode",  false);
        }
    }
);
"""

dash_css = """/****** DASHBOARD BUTTONS ******/
.jupyter-dashboard-toolbar-buttons {
    visibility: hidden;
}
"""

main_css = """/****** AUTOMATICALLY GENERATED BY NotebookSettings **************/

/* Warning: Anything you add here will get overwritten! */

/* remove clusters tab in tree view */
.clusters_tab_link {
    visibility: hidden;
}
/* remove CellToobar Reminder */
[title="show new celltoolbar selector location"] {
    display: none;
}

/* FULL WIDTH */
.notebook_app .container {
    width:99%; !important
}

/* Prevent the edit cell highlight box from getting clipped;
 * important so that it also works when cell is in edit mode*/
div.cell.selected {
    border-left-width: 1px !important;
}
"""

eqn_js = """/** EQUATION NUMBERING **/
require([
    'base/js/namespace',
    'jquery'
],   function(IPython, $) {
    "use strict";

    IPython.toolbar.add_buttons_group([
        {
            id: 'reset_numbering',
            label: 'Reset equation numbering',
            icon: 'fa-sort-numeric-asc text-primary',
            callback: function () {
                MathJax.Hub.Queue(
                    ["resetEquationNumbers", MathJax.InputJax.TeX],
                    ["PreProcess", MathJax.Hub],
                    ["Reprocess", MathJax.Hub]
                );
                $('#reset_numbering').blur();
            }
        }
    ]);
    MathJax.Hub.Config({
        TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
});
"""

header_template = """
/** HEADER 4 **/
require(["base/js/namespace"],
    function(Jupyter) {
        var logo = document.getElementById("ipython_notebook").children[0];
        var host = window.location.hostname.replace('proxy.', '');
        var uid = ${uid};
        try {var base = Jupyter.notebook.base_url;}
        catch (err) {var base = IPython.notebook.base_url;}
        var stat = 'https://' + window.location.hostname + base + 'static/';
        logo.href = 'https://' + host + '/members/' + uid + '/dashboard';
        logo.target = "_blank";
        logo.title = "${shost} Dashboard";
        var hostlogo = stat + '${lhost}.png';
        var jlogo = stat + 'jupyter.png';
        logo.innerHTML=`<img src="${hostlogo}" style="margin:0px 20px 0px 0px"><img src="${jlogo}">`;

        var term = document.getElementById("login_widget");
        var end = base.indexOf('/',7);
        var session = base.slice(7,end);
        var url = `https://${host}/tools/jupyter/stop?sess=${session}`
        term.innerHTML = `<button class="btn btn-sm navbar-btn" title="Terminate this notebook or tool and any others in the session" onclick="window.location.href='${url}'">Terminate Session</button>`;
    }
);
"""

In [5]:
def update_css():
    global current_dash
    with open(css_filename, 'w') as f:
        f.write(main_css)
    if current_dash == False:
        with open(css_filename, 'a') as f:
            f.write(dash_css)
    print("Please reload any open notebooks for the changes to take effect.")

def set_editor(km):
    global current_eqn, current_editor
    current_editor = km
    vimmode = str(km == 'vim').lower()
    
    # write edit.json
    with open(edit_filename, 'w') as f:
        f.write(edit_config % (vimmode, km))

    # write custom.js
    with open(cjs_filename, 'w') as f:
        if km == 'emacs':
            f.write(emacs_editor_txt)
        else:
            f.write(sublime_editor_txt)
        t = Template(header_template)
        host = os.uname()[1]
        uid = os.getuid()
        f.write(t.safe_substitute(uid=uid, shost=host, lhost=host.lower()))
        
    # equation numbering
    if current_eqn:
        with open(cjs_filename, 'a') as f:
            f.write(eqn_js)
    print("Please reload any open notebooks for the changes to take effect.")

def set_eqn(new):
    global current_eqn, current_editor
    current_eqn = new
    set_editor(current_editor)
    print("Please reload any open notebooks for the changes to take effect.")

def set_doc_tools(new):
    global current_doc
    current_doc = new
    cm.update('notebook', {"load_extensions": {doc_tools_path: current_doc}})
    print("Please reload any open notebooks for the changes to take effect.")

def set_dash_tools(new):
    global current_dash
    current_dash = new
    update_css()

In [6]:
# create user config dirs if none exists
if not os.path.exists(cdir):
    os.system('mkdir -p ' + cdir)
if not os.path.exists(ndir):
    os.system('mkdir -p ' + ndir)

# read current values
current_editor = None
current_eqn = False
try:
    f = open(cjs_filename, 'r')
    for line in f.read().split('\n'):
        if line.startswith('/** KEYMAP '):
            current_editor = line.split()[2]
        elif line.startswith('/** EQUATION NUMBERING'):
            current_eqn = True
    f.close()
except:
    pass

if current_editor is None:
    current_editor = 'emacs'
    set_editor(current_editor)

current_dash = True
try:
    f = open(css_filename, 'r')
    for line in f.read().split('\n'):
        if line.startswith("/****** DASH"):
            current_dash = False
    f.close()
except:
    pass

try:
    current_doc = cm.get('notebook')['load_extensions'][doc_tools_path]
except:
    cm.update('notebook', {"load_extensions": {doc_tools_path: False}})
    current_doc = False

# Notebook Settings

## Editor Mode
The notebook's built-in editor can emulate other popular editors.  

In [7]:
create_form("Keymap", {"Emacs": "emacs", "Sublime": "sublime"}, current_editor, set_editor)

Box(children=(Label(value=u'Keymap', layout=Layout(flex=u'1 1 auto')), Dropdown(layout=Layout(flex=u'0 1 10%'), options={'Emacs': 'emacs', 'Sublime': 'sublime'}, value='emacs')), layout=Layout(border=u'solid 1px lightgray', display=u'flex', flex_flow=u'row', justify_content=u'space-between', width=u'60%'))

## Dashboard Tools

<img  src="dashboard.png" style="margin: 0"/>
Enables and disables the display of the dashboard tool buttons.  You will only need
these if you are creating dashboards (tools) instead of notebooks.

In [8]:
create_form("Dashboard Tools", {"OFF": False, "ON": True}, current_dash, set_dash_tools)

Box(children=(Label(value=u'Dashboard Tools', layout=Layout(flex=u'1 1 auto')), Dropdown(layout=Layout(flex=u'0 1 10%'), options={'ON': True, 'OFF': False}, value=True)), layout=Layout(border=u'solid 1px lightgray', display=u'flex', flex_flow=u'row', justify_content=u'space-between', width=u'60%'))

## Document Tools

<img  src="document_tools2.png" style="margin: 0"/>
If you want your notebook sections numbered and indexed, with references at the end, you should try this.  

Adds buttons to your menubar to move sections up and down, automatically number sections, generate a table of contents and generate references.

In [9]:
create_form("Document Tools", {"OFF": False, "ON": True}, current_doc, set_doc_tools)

Box(children=(Label(value=u'Document Tools', layout=Layout(flex=u'1 1 auto')), Dropdown(index=1, layout=Layout(flex=u'0 1 10%'), options={'ON': True, 'OFF': False}, value=False)), layout=Layout(border=u'solid 1px lightgray', display=u'flex', flex_flow=u'row', justify_content=u'space-between', width=u'60%'))

## Equation Numbering

This will add an icon <img  src="renumber.png" style="margin: 0"/>
to the menubar which will renumber equations and generate references when clicked.
See [Equation Numbering](../manual/markdown.ipynb#Equation-Numbering-and-References) in the manual for more information.

In [10]:
create_form("Equation Numbering", {"OFF": False, "ON": True}, current_eqn, set_eqn)

Box(children=(Label(value=u'Equation Numbering', layout=Layout(flex=u'1 1 auto')), Dropdown(index=1, layout=Layout(flex=u'0 1 10%'), options={'ON': True, 'OFF': False}, value=False)), layout=Layout(border=u'solid 1px lightgray', display=u'flex', flex_flow=u'row', justify_content=u'space-between', width=u'60%'))

In [11]:
input_form = """
<p>Clicking 'DONE' will exit this tool.  You should confirm if prompted by your browser.</p>
<hr>
<center><button id="stop_button" style="font-size: 24px;">DONE</button></center>
"""

js = """
<script type="text/Javascript">
$(document).ready(
function() {
    $("#stop_button").click(function() {
        var close_window = function () {
            /**
             * allow closing of new tabs in Chromium, impossible in FF
            */
            window.open('', '_self', '');
            window.close();
        };
        IPython.notebook.session.delete(close_window, close_window);
        });
    });
</script>
"""
display(HTML(input_form + js))